# TP2 - Sistema RAG de Recomendación de Destinos Turísticos

**Inteligencia Artificial - Ingeniería en Sistemas de Información**

Sistema de recomendación de destinos turísticos europeos usando RAG (Retrieval-Augmented Generation).

**Stack:** Langchain + LLaMA 3 (Groq) + HuggingFace Embeddings + ChromaDB

## 1. Instalación de dependencias

In [ ]:
# Instalar librerías necesarias
!pip install -q langchain langchain-groq langchain-huggingface langchain-community
!pip install -q chromadb pandas python-dotenv sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 2. Configuración

Obtener API key gratuita en: https://console.groq.com

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

# Configurar API key de Groq
os.environ['GROQ_API_KEY'] = "PEGAR_KEY_ACÁ"

# Rutas
CSV_PATH = "european_destinations.csv"  # El CSV debe estar en la misma carpeta
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

# Verificación
if not GROQ_API_KEY or 'tu_key' in GROQ_API_KEY:
    print("⚠️ Configura tu API key de Groq arriba")
else:
    print("✅ Configuración OK")

✅ Configuración OK


## 3. Carga del dataset

Dataset de 209 destinos turísticos europeos de Kaggle.

In [ ]:
import pandas as pd

def load_european_dataset(csv_path):
    """Carga el CSV con encoding latin1 para caracteres especiales"""
    print("Cargando dataset...")
    df = pd.read_csv(csv_path, encoding='latin1')
    print(f"✅ {len(df)} destinos de {df['Country'].nunique()} países cargados")
    return df

# Cargar dataset
df = load_european_dataset(CSV_PATH)

# Ver primeras filas
df.head(3)

Cargando dataset...
✅ 209 destinos de 21 países cargados


,Destination,Region,Country,Category,Latitude,Longitude,Approximate Annual Tourists,Currency,Majority Religion,Famous Foods,Language,Best Time to Visit,Cost of Living,Safety,Cultural Significance,Description
0,Rome,Lazio,Italy,City,41.902782,12.496366,14 million,Euro,Roman Catholic,"Pizza, Pasta, Gelato",Italian,Spring (April-May) or Fall (Sept-Oct),Medium-high,"Generally safe, but watch out for pickpockets","The capital city, known for its historical lan...","A hub of ancient history and modern culture, w..."
1,Florence,Tuscany,Italy,City,43.769581,11.255772,10 million,Euro,Roman Catholic,"Pizza, Pasta, Gelato",Italian,Spring (April-May) or Fall (Sept-Oct),Medium-high,"Generally safe, but watch out for pickpockets","A Renaissance city famous for its art, archite...","Home to world-class museums, including the Uff..."
2,Venice,Veneto,Italy,City,45.435559,12.336196,10 million,Euro,Roman Catholic,"Pizza, Pasta, Gelato",Italian,Spring (April-May) or Fall (Sept-Oct),Medium-high,"Generally safe, but watch out for pickpockets","A unique city built on canals, known for its g...","An iconic city of water, renowned for romantic..."


### Exploración del dataset

In [ ]:
def explore_dataset(df):
    """Estadísticas básicas del dataset"""
    print(f"Total destinos: {len(df)}\n")

    print("Top 5 países:")
    print(df['Country'].value_counts().head())

    print("\nTop 10 categorías:")
    print(df['Category'].value_counts().head(10))

    print("\nNiveles de presupuesto:")
    print(df['Cost of Living'].value_counts())

    return df

df = explore_dataset(df)

Total destinos: 209

Top 5 países:
Country
Italy      10
Spain      10
France     10
Austria    10
Belgium    10
Name: count, dtype: int64

Top 10 categorías:
Category
City                   91
Town                   25
Region                 12
Island                 11
National Park          10
Garden                  4
Castle                  3
Beach                   3
Coastal City            3
Archaeological Site     3
Name: count, dtype: int64

Niveles de presupuesto:
Cost of Living
Medium-high       107
Medium             72
High               15
Free               10
Extremely high      4
Varies              1
Name: count, dtype: int64


## 4. Preparación de datos para RAG

Creamos descripciones enriquecidas combinando múltiples columnas del dataset. Esto es lo que se va a embedear.

In [ ]:
def create_rag_descriptions(df):
    """
    Combina múltiples columnas en una descripción única para cada destino.

    ¿Por qué hacemos esto?
    - Los embeddings funcionan mejor con texto continuo que con columnas separadas
    - Consolidamos toda la info relevante en un solo campo
    - Este texto es lo que se va a convertir en vectores para la búsqueda
    """
    print("Creando descripciones enriquecidas...")

    def build_description(row):
        # Tomamos datos de múltiples columnas y armamos un texto estructurado
        # Formato consistente ayuda al modelo a entender la info
        description = f"""
DESTINATION: {row['Destination']}, {row['Country']} ({row['Region']})
TYPE: {row['Category']}

DESCRIPTION: {row.get('Description', '')}
CULTURAL SIGNIFICANCE: {row.get('Cultural Significance', '')}

BEST TIME TO VISIT: {row.get('Best Time to Visit', 'Year-round')}
BUDGET: {row.get('Cost of Living', 'Medium')}
ANNUAL TOURISTS: {row.get('Approximate Annual Tourists', 'Popular destination')}

PRACTICAL INFO:
- Language: {row.get('Language', 'Local language')}
- Currency: {row.get('Currency', 'Euro')}
- Safety: {row.get('Safety', 'Generally safe')}
- Famous for: {row.get('Famous Foods', 'Local cuisine')}

COORDINATES: {row.get('Latitude', 0)}, {row.get('Longitude', 0)}
        """.strip()
        return description

    # apply() ejecuta la función en cada fila del DataFrame
    df['rag_description'] = df.apply(build_description, axis=1)
    print(f"✅ {len(df)} descripciones creadas")
    return df

df = create_rag_descriptions(df)

# Ver ejemplo para entender qué generamos
print("\nEjemplo (Rome):")
print("-" * 60)
print(df[df['Destination'] == 'Rome']['rag_description'].iloc[0])

Creando descripciones enriquecidas...
✅ 209 descripciones creadas

Ejemplo (Rome):
------------------------------------------------------------
DESTINATION: Rome, Italy (Lazio)
TYPE: City

DESCRIPTION: A hub of ancient history and modern culture, with rich traditions, art, and landmarks.
CULTURAL SIGNIFICANCE: The capital city, known for its historical landmarks like the Colosseum, Vatican City, and Pantheon.

BEST TIME TO VISIT: Spring (April-May) or Fall (Sept-Oct)
BUDGET: Medium-high
ANNUAL TOURISTS: 14 million

PRACTICAL INFO:
- Language: Italian
- Currency: Euro
- Safety: Generally safe, but watch out for pickpockets
- Famous for: Pizza, Pasta, Gelato

COORDINATES: 41.902782, 12.496366


## 5. Conversión a Documents de Langchain

Convertimos el DataFrame a objetos Document con page_content (texto a embedear) y metadata (info adicional).

In [ ]:
from langchain.docstore.document import Document

def create_documents_from_dataframe(df):
    """
    Convierte DataFrame a formato Document de Langchain.

    Document tiene dos partes:
    - page_content: El texto que se va a embedear (nuestra descripción)
    - metadata: Info adicional que NO se embedea pero sirve para filtros

    ¿Por qué separamos contenido y metadata?
    - El embedding solo captura semántica del contenido
    - Los metadatos se usan para mostrar resultados y aplicar filtros
    """
    print("Convirtiendo a Documents...")

    documents = []
    for idx, row in df.iterrows():
        doc = Document(
            # Lo que se va a buscar por similitud semántica, solo aplicamos embedding sobre la columna rag_description
            page_content=row['rag_description'],

            # Info estructurada para usar después (no se embedea)
            metadata={
                'destination': row['Destination'],
                'country': row['Country'],
                'category': row['Category'],
                'budget': row.get('Cost of Living', 'Medium'),
                'best_time': row.get('Best Time to Visit', 'Year-round')
            }
        )
        documents.append(doc)

    print(f"✅ {len(documents)} documentos creados")
    return documents

documents = create_documents_from_dataframe(df)

print("\nEstructura de un Document:")
print(f"- page_content: {documents[0].page_content[:100]}...")
print(f"- metadata: {documents[0].metadata}")

Convirtiendo a Documents...
✅ 209 documentos creados

Estructura de un Document:
- page_content: DESTINATION: Rome, Italy (Lazio)
TYPE: City

DESCRIPTION: A hub of ancient history and modern cultur...
- metadata: {'destination': 'Rome', 'country': 'Italy', 'category': 'City', 'budget': 'Medium-high', 'best_time': 'Spring (April-May) or Fall (Sept-Oct)'}


## 6. Embeddings

Modelo de HuggingFace que convierte texto en vectores de 384 dimensiones para búsqueda semántica.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import sys
import os

print("Cargando modelo de embeddings...")
print("(Primera vez descarga ~80MB del modelo)")

# Suprimir errores de protobuf temporalmente
stderr_backup = sys.stderr
sys.stderr = open(os.devnull, 'w')

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Restaurar stderr
sys.stderr = stderr_backup

print("✅ Modelo cargado")

# Demostración
test_text = "ciudad histórica con museos"
test_embedding = embeddings.embed_query(test_text)

print(f"\nTexto: '{test_text}'")
print(f"Se convierte en: vector de {len(test_embedding)} números")
print(f"Primeros 5 valores: {test_embedding[:5]}")
print("\nTextos similares tienen vectores cercanos (distancia coseno)")

Cargando modelo de embeddings...
(Primera vez descarga ~80MB del modelo)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Modelo cargado

Texto: 'ciudad histórica con museos'
Se convierte en: vector de 384 números
Primeros 5 valores: [0.022915363311767578, 0.08624137192964554, -0.014795925468206406, 0.02384246326982975, -0.04984007403254509]

Textos similares tienen vectores cercanos (distancia coseno)


## 7. Vector Store (ChromaDB)

Base de datos vectorial que almacena los embeddings y permite búsqueda por similitud.

In [ ]:
from langchain_community.vectorstores import Chroma
import shutil
import os

print("Creando vector store...")

# IMPORTANTE: Borrar vector store anterior si existe (evita duplicados)
if os.path.exists("vectordb"):
    shutil.rmtree("vectordb")
    print("Vector store anterior eliminado")

print("Proceso: convierte cada documento a embedding y los guarda en ChromaDB")
print("(Tarda 1-2 min porque son 209 documentos)")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory="vectordb"
)

print(f"✅ Vector store creado: {vectorstore._collection.count()} docs indexados")

# Prueba de búsqueda
print("\nPrueba de similarity search:")
test_query = "playa tranquila"
results = vectorstore.similarity_search(test_query, k=3)
print(f"Query: '{test_query}'")
print("Los 3 docs más similares:")
for i, doc in enumerate(results, 1):
    print(f"  {i}. {doc.metadata['destination']}, {doc.metadata['country']}")

Creando vector store...
Proceso: convierte cada documento a embedding y los guarda en ChromaDB
(Tarda 1-2 min porque son 209 documentos)
✅ Vector store creado: 209 docs indexados

Prueba de similarity search:
Query: 'playa tranquila'
Los 3 docs más similares:
  1. Cinque Terre, Italy
  2. Subotica, Serbia
  3. Vrnja?ka Banja, Serbia


## 8. Retriever personalizado con diversidad geográfica

### Problema detectado

El similarity search simple devolvía múltiples destinos del mismo país:
- Query: "ciudad histórica" → Madrid, Sevilla, Barcelona (todos España)
- Causa: España está sobre-representado en el dataset (10 destinos vs otros países)

### Solución implementada

Creamos un retriever personalizado que:
1. Busca 50 documentos candidatos por similitud
2. Filtra para tener **máximo 1 destino por país**
3. Devuelve los 8 más relevantes con diversidad garantizada

Esto mejora la experiencia del usuario dando variedad de opciones.

In [ ]:
class DiversityRetriever:
    """
    Retriever personalizado que fuerza diversidad geográfica.

    Flujo:
    1. Busca 50 candidatos por similitud (más de los que necesitamos)
    2. Itera sobre ellos en orden de relevancia
    3. Toma el primero de cada país (máximo 1 por país)
    4. Para cuando tiene k=8 documentos diversos
    """
    def __init__(self, vectorstore, k=8):
        self.vectorstore = vectorstore
        self.k = k  # Cuántos documentos devolver al final

    def get_relevant_documents(self, query):
        # Paso 1: Buscar muchos candidatos (50 en lugar de 8)
        docs = self.vectorstore.similarity_search(query, k=50)

        # Paso 2: Filtrar por diversidad
        seen_countries = set()  # Países ya incluidos
        diverse_docs = []       # Resultado final

        for doc in docs:
            country = doc.metadata.get('country')

            # Solo agregamos si el país no fue visto antes
            if country not in seen_countries:
                seen_countries.add(country)
                diverse_docs.append(doc)

            # Paramos cuando tenemos suficientes docs diversos
            if len(diverse_docs) >= self.k:
                break

        return diverse_docs

# Crear retriever con diversidad
retriever = DiversityRetriever(vectorstore, k=8)

# Comparación: antes vs después
print("Sin filtro de diversidad:")
normal_docs = vectorstore.similarity_search("ciudad histórica", k=5)
print([d.metadata['country'] for d in normal_docs])

print("\nCon filtro de diversidad:")
diverse_docs = retriever.get_relevant_documents("ciudad histórica")
print([d.metadata['country'] for d in diverse_docs[:5]])

Sin filtro de diversidad:
['Spain', 'Spain', 'Spain', 'Spain', 'Spain']

Con filtro de diversidad:
['Spain', 'Italy', 'Serbia', 'Portugal', 'Malta']


##9. Analizador de contextos
Detecta automáticamente si el usuario sigue hablando del mismo tema o cambió


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np

class ContextController:
    """
    Controla el contexto conversacional de manera semántica.

    Mantiene un historial breve y filtra según coherencia.
    Si el usuario cambia de tema (semánticamente distinto),
    reinicia el historial.
    """

    def __init__(self, embedding_model, similarity_threshold=0.38, max_history=3):
        """
        embedding_model: el mismo objeto usado en el vector store
        similarity_threshold: similitud mínima para considerar misma intención
        max_history: cuántas consultas coherentes mantener
        """
        self.embedding_model = embedding_model
        self.similarity_threshold = similarity_threshold
        self.max_history = max_history
        self.history = []  # guarda las últimas consultas coherentes
        self.last_embedding = None


    def cosine_similarity(self, v1, v2):
        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

    def update_and_generate(self, new_query: str):
        """
        Analiza la nueva consulta y decide:
        - Si es coherente con la anterior → combina consultas
        - Si no → limpia historial
        Devuelve el texto final que se enviará al RAG.
        """
        new_emb = self.embedding_model.embed_query(new_query)

        # Primer turno: no hay contexto previo
        if self.last_embedding is None:
            self.history = [new_query]
            self.last_embedding = new_emb
            return new_query

        # Calcular similitud con la consulta previa
        sim = self.cosine_similarity(self.last_embedding, new_emb)

        # Si la similitud es baja, nuevo tema → resetear
        if sim < self.similarity_threshold:
            self.history = [new_query]
            self.last_embedding = new_emb
            print(f"[ContextController] 🔄 Nuevo tema detectado (sim={sim:.2f}) → contexto reiniciado")
            return new_query

        # Si es coherente, mantener historial breve y fusionar
        self.history.append(new_query)
        self.history = self.history[-self.max_history:]
        self.last_embedding = new_emb

        combined_query = " ".join(self.history)
        print(f"[ContextController] ✅ Contexto coherente (sim={sim:.2f}) → query combinada")
        return combined_query


# Crear instancia global del controlador de contexto
context_controller = ContextController(embedding_model=embeddings, similarity_threshold=0.38,  max_history=3)


print("✅ Context Controller listo")
print(context_controller.similarity_threshold)



✅ Context Controller listo
0.38


## 10. Prompt Engineering

Prompt estricto para que LLaMA solo use información de los documentos recuperados y no invente destinos.

In [ ]:
from langchain.prompts import PromptTemplate

# Prompt engineering: las instrucciones son críticas para la calidad
#
# Problema identificado: LLaMA a veces agregaba info de su conocimiento general
# Ejemplo: mencionaba "senderismo" aunque no estuviera en la descripción
#
# Solución: Prompt EXTREMADAMENTE estricto

template = """Eres un asistente de viajes. Tu ÚNICA función es analizar la información proporcionada abajo y recomendarla al usuario.

REGLAS ABSOLUTAS:
1. SOLO usa información que esté LITERALMENTE escrita en el contexto de abajo
2. NO agregues detalles aunque parezcan lógicos o probables
3. NO uses tu conocimiento general sobre geografía o turismo
4. Si la información no menciona algo específicamente, NO lo menciones
5. Copia/parafrasea únicamente lo que está escrito

Información de destinos:
{context}

Pregunta: {question}

Recomienda 2-3 destinos basándote ÚNICAMENTE en lo escrito arriba.
Para cada destino, menciona SOLO lo que aparece en su descripción:
- Por qué es apropiado (según lo que dice la descripción)
- Época de visita (exactamente como está escrito)
- Presupuesto (exactamente como está escrito)

NO agregues actividades, características o detalles que no estén mencionados explícitamente.

Respuesta:"""

# PromptTemplate permite inyectar variables {context} y {question} dinámicamente
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

print("✅ Prompt configurado")
print("\nEl prompt tiene dos variables que se rellenan automáticamente:")
print("- context: Los documentos que encuentra el retriever")
print("- question: La pregunta del usuario")

✅ Prompt configurado

El prompt tiene dos variables que se rellenan automáticamente:
- context: Los documentos que encuentra el retriever
- question: La pregunta del usuario


## 11. Configuración del LLM y cadena RAG completa

LLaMA 3 (70B parámetros) via Groq API + cadena QA que integra retriever + prompt + LLM.

In [ ]:
from langchain_groq import ChatGroq
from langchain.chains.question_answering import load_qa_chain

print("Ensamblando sistema RAG completo...")

# Paso 1: Configurar LLM
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",  # LLaMA 3 de 70B parámetros
    api_key=GROQ_API_KEY,
    temperature=0.3  # 0=determinista, 1=creativo. 0.3 es buen balance
)

# Paso 2: Crear cadena QA
# "stuff" significa: meter todos los docs en el prompt de una vez
qa_chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)

# Paso 3: Wrapper que integra todo
class RAGSystem:
    """
    Sistema RAG completo que conecta:
    - Retriever (busca docs relevantes)
    - QA Chain (LLM genera respuesta usando esos docs)

    Flujo al hacer una consulta:
    1. Usuario pregunta
    2. Retriever busca en vector store → devuelve 8 docs diversos
    3. Docs se insertan en {context} del prompt
    4. LLM lee prompt + contexto + pregunta
    5. LLM genera respuesta basada en los docs
    """
    def __init__(self, retriever, qa_chain):
        self.retriever = retriever
        self.qa_chain = qa_chain

    def invoke(self, inputs):
        query = inputs['query']

        # Paso 1: Retrieval - buscar documentos relevantes
        docs = self.retriever.get_relevant_documents(query)

        # Paso 2: Generation - LLM genera respuesta usando esos docs
        result = self.qa_chain({"input_documents": docs, "question": query})

        # Devolver respuesta + docs usados (para transparencia)
        return {
            'result': result['output_text'],
            'source_documents': docs
        }

# Crear sistema
rag_system = RAGSystem(retriever, qa_chain)

print("✅ Sistema RAG ensamblado")
print("\nComponentes integrados:")
print("1. DiversityRetriever → busca docs diversos")
print("2. Prompt estricto → evita alucinaciones")
print("3. LLaMA 3 → genera respuestas coherentes")

Ensamblando sistema RAG completo...
✅ Sistema RAG ensamblado

Componentes integrados:
1. DiversityRetriever → busca docs diversos
2. Prompt estricto → evita alucinaciones
3. LLaMA 3 → genera respuestas coherentes


/tmp/ipython-input-1423935443.py:15: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)


## 12. Función de consulta

Función helper para ejecutar consultas y mostrar resultados.

In [ ]:
def query_rag_system(rag_system, user_query):
    """Ejecuta consulta y muestra resultados formateados"""
    print(f"\n{'='*70}")
    print(f"CONSULTA ORIGINAL: {user_query}")
    print('='*70)

    # 🔍 Analizar coherencia y generar query final
    query_final = context_controller.update_and_generate(user_query)
    print(f"\nQUERY FINAL USADA: {query_final}")

    # Ejecutar con query procesada
    result = rag_system.invoke({"query": query_final})

    # Mostrar respuesta
    print("\nRESPUESTA:")
    print(result['result'])

    # Mostrar documentos recuperados
    print(f"\n{'='*70}")
    print("DOCUMENTOS RECUPERADOS:")
    print('='*70)
    for i, doc in enumerate(result['source_documents'][:5], 1):
        print(f"{i}. {doc.metadata['destination']}, {doc.metadata['country']}")
        print(f"   Tipo: {doc.metadata['category']} | Presupuesto: {doc.metadata['budget']}")

    return result


## 13. Casos de prueba

Probamos el sistema con diferentes tipos de consultas.

In [ ]:
# Caso 1: Destino donde haya montañas
resultado_1 = query_rag_system(
    rag_system,
    "Quiero un destino donde haya montañas"
)


CONSULTA ORIGINAL: Quiero un destino donde haya montañas

QUERY FINAL USADA: Quiero un destino donde haya montañas


/tmp/ipython-input-1423935443.py:42: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain({"input_documents": docs, "question": query})



RESPUESTA:
Basándome en la información proporcionada, te recomiendo los siguientes destinos con montañas:

1. **French Alps, France (Auvergne-Rhône-Alpes)**
 * Por qué es apropiado: "A haven for outdoor enthusiasts, famous for its ski resorts and stunning alpine scenery."
 * Época de visita: "Winter (Dec-Mar) for skiing, Summer (Jun-Sept)"
 * Presupuesto: "Medium-high"

2. No hay otros destinos que mencionen explícitamente la presencia de montañas en su descripción. Sin embargo, puedo mencionar que algunos destinos tienen una ubicación geográfica que podría sugerir la presencia de montañas, pero no está explícitamente mencionado en la descripción.

Si deseas considerar otros destinos, puedo mencionar que **Meteora, Greece (Thessaly)** tiene una descripción que menciona "monasteries on rock formations", lo que podría sugerir una ubicación montañosa, pero no está explícitamente mencionado.

* Por qué es apropiado: No se menciona explícitamente la presencia de montañas, pero se menciona 

In [ ]:
# Caso 2: Concatenacion 2--> Lenguaje nativo italiano
resultado_2 = query_rag_system(
    rag_system,
    "Y que el lenguaje nativo sea italiano"
)


CONSULTA ORIGINAL: Y que el lenguaje nativo sea italiano
[ContextController] ✅ Contexto coherente (sim=0.48) → query combinada

QUERY FINAL USADA: Quiero un destino donde haya montañas Y que el lenguaje nativo sea italiano

RESPUESTA:
Basándome en la información proporcionada, te recomiendo el siguiente destino que cumple con tus requisitos:

* Lake Como, Italy (Lombardy)
 + Por qué es apropiado: "A luxury retreat for the elite, featuring serene waters and mountain landscapes."
 + Época de visita: Spring (April-May) o Fall (Sept-Oct)
 + Presupuesto: Medium-high

No hay otros destinos que mencionen explícitamente la presencia de montañas y el idioma italiano como lengua nativa. Sin embargo, es importante destacar que la información proporcionada es limitada y no se mencionan otros posibles destinos que podrían cumplir con tus requisitos.

DOCUMENTOS RECUPERADOS:
1. M laga, Spain
   Tipo: City | Presupuesto: Medium
2. Lake Como, Italy
   Tipo: Lake | Presupuesto: Medium-high
3. Braga, P

In [ ]:
# Caso 3: Concatenacion 3--> lugar donde haya playas y montañas y hablen italiano
resultado_3 = query_rag_system(
    rag_system,
    "Necesito un lugar donde haya playas"
)


CONSULTA ORIGINAL: Necesito un lugar donde haya playas
[ContextController] ✅ Contexto coherente (sim=0.39) → query combinada

QUERY FINAL USADA: Quiero un destino donde haya montañas Y que el lenguaje nativo sea italiano Necesito un lugar donde haya playas

RESPUESTA:
Basándome en la información proporcionada, te recomiendo los siguientes destinos:

1. **Lake Como, Italy (Lombardy)**
 * Por qué es apropiado: A beautiful lake surrounded by mountains
 * Época de visita: Spring (April-May) o Fall (Sept-Oct)
 * Presupuesto: Medium-high
2. No hay otros destinos que cumplan con todos los requisitos mencionados (montañas, lenguaje nativo italiano y playas). Sin embargo, Lake Como es un destino que menciona montañas y lenguaje italiano, pero no playas explícitamente. Si se considera que un lago puede ser similar a una playa, entonces Lake Como podría ser una opción.
 
No hay suficiente información para recomendar otros destinos que cumplan con todos los requisitos.

DOCUMENTOS RECUPERADOS:
1.

In [ ]:
# Caso 4: Consulta personalizada--> reinicio contexto otra vez
mi_consulta = "quiero conocer el vaticano"  # Modificar y ejecutar
resultado_4 = query_rag_system(rag_system, mi_consulta)


CONSULTA ORIGINAL: quiero conocer el vaticano
[ContextController] 🔄 Nuevo tema detectado (sim=0.36) → contexto reiniciado

QUERY FINAL USADA: quiero conocer el vaticano

RESPUESTA:
Basándome en la información proporcionada, te recomiendo los siguientes destinos para conocer el Vaticano:

1. **Roma, Italia**: 
 - Es apropiado porque es conocida por sus "landmarks como el Colosseum, Vatican City, y Pantheon".
 - La mejor época para visitar es: "Spring (April-May) o Fall (Sept-Oct)"
 - El presupuesto es: "Medium-high"

2. **Oceanographic Museum, Monaco**: 
 - No se menciona el Vaticano explícitamente, pero como no hay más opciones directas, se puede considerar por su ubicación en Europa, aunque no es directamente relacionado con el Vaticano.
 - La mejor época para visitar es: "Spring (April-May) o Fall (Sept-Oct)"
 - El presupuesto es: "Medium"

3. No hay un tercer destino que se relacione directamente con el Vaticano según la información proporcionada.

DOCUMENTOS RECUPERADOS:
1. Madrid


Sistema RAG funcional implementado con:
- ✅ Dataset de 209 destinos procesado
- ✅ Embeddings y vector store creados
- ✅ Retriever con diversidad geográfica
- ✅ LLaMA 3 integrado via Groq
- ✅ Prompt engineering para evitar alucinaciones

## Limitaciones y dificultades identificadas

### 1. Retrieval por embeddings no garantiza keywords exactos

**Problema:** Query "quiero comer pizza y gelato" no recupera Rome aunque tiene ambos en su descripción.

**Causa:**
- Embeddings capturan similitud semántica general, no hacen matching exacto de palabras
- Dan más peso a términos prominentes en la descripción principal
- Rome tiene "Pizza, Pasta, Gelato" al final en "Famous for:", menos prominente

**Evidencia:**
- Rome: "Famous for: Pizza, Pasta, Gelato" (posición baja en el texto)
- Naples: "Famous for pizza" (en descripción principal)
- Retriever prioriza Naples

**Mejora futura:** Búsqueda híbrida (embeddings + keyword search)

---

### 2. Caracteres especiales con encoding

**Problema:** Algunos nombres aparecen con `?` en lugar de caracteres especiales (Óbidos → ?bidos, Vrnjaĉka → Vrnja?ka)

**Causa:** CSV usa caracteres UTF-8 pero se lee con latin1 para compatibilidad

**Impacto:** Cosmético, no afecta funcionalidad del RAG

---

### 3. Sesgo geográfico del dataset

**Problema inicial:** Sistema recomendaba principalmente destinos españoles

**Causa:** España sobre-representada en dataset (10 destinos vs otros países)

**Solución implementada:** DiversityRetriever que garantiza máximo 1 destino por país

**Resultado:** Problema resuelto, ahora hay diversidad geográfica

---

### 4. Warnings de protobuf

**Problema:** Errores "AttributeError: 'MessageFactory'" al cargar embeddings

**Causa:** Conflicto entre versiones de protobuf (tensorflow/mediapipe vs langchain)

**Impacto:** Solo visual, no impide funcionamiento del sistema

---
### 5. Limitaciones del context controller
**Umbral empírico**: El valor de similarity_threshold se ajustó mediante pruebas; puede no ser óptimo para todas las consultas, provocando combinaciones inesperadas o reinicios innecesarios del historial.

**Similitud semántica imperfecta**: Frases con palabras comunes o estructuras similares pueden resultar en valores de similitud altos aunque sean temas distintos, generando concatenaciones de queries no coherentes.

**Historial limitado**: Solo se mantienen las últimas max_history consultas, por lo que información de consultas más antiguas puede perderse en diálogos largos

## Resultados obtenidos

- Sistema RAG funcional con 209 destinos europeos
- Respuestas coherentes basadas en datos reales del dataset
- Diversidad geográfica en recomendaciones
- Prompt engineering efectivo para evitar alucinaciones mayores
- Limitaciones documentadas y comprendidas